In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

import re
import sys
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# ===========================================================================
# 1) Provide the input markdown.
# ===========================================================================
MARKDOWN_TEXT = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

# ===========================================================================
# 2) Authenticate to Google Docs from Colab.
# ===========================================================================
try:
    auth.authenticate_user()
    docs_service = build("docs", "v1")
    print("Successfully authenticated with Google Docs API!")
except Exception as e:
    print("Error during authentication:", str(e))
    sys.exit(1)

# ===========================================================================
# 3) Helper functions for building requests in forward order.
# ===========================================================================
def create_new_document(title):
    """
    Creates a new Google Doc with the specified title.
    Returns the created document ID.
    """
    try:
        doc = docs_service.documents().create(body={"title": title}).execute()
        print(f"Created document with title: {doc.get('title')}")
        return doc.get("documentId")
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

def extract_nesting_level(line):
    """
    Count the number of leading spaces to determine nesting level.
    Each 2 spaces = 1 nesting level.
    """
    leading_spaces = len(line) - len(line.lstrip(" "))
    return leading_spaces // 2

def build_insert_text_request(text, current_index):
    """
    Insert `text` at current_index. Returns (request, new_index).
    The new_index = current_index + len(text) + 1 (for the newline).
    """
    return {
        "insertText": {
            "location": {"index": current_index},
            "text": text + "\n"
        }
    }, current_index + len(text) + 1  # +1 for the newline

def build_update_paragraph_style_request(start_index, end_index, style_type):
    """
    Updates the paragraph style (e.g. HEADING_1, HEADING_2, HEADING_3, NORMAL_TEXT).
    """
    return {
        "updateParagraphStyle": {
            "range": {
                "startIndex": start_index,
                "endIndex": end_index
            },
            "paragraphStyle": {
                "namedStyleType": style_type
            },
            "fields": "namedStyleType"
        }
    }

def build_bullet_request(start_index, end_index, checkbox=False):
    """
    Turns the paragraph in [start_index, end_index) into a bullet or checkbox bullet.
    """
    return {
        "createParagraphBullets": {
            "range": {
                "startIndex": start_index,
                "endIndex": end_index
            },
            "bulletPreset": "BULLET_CHECKBOX" if checkbox else "BULLET_DISC_CIRCLE_SQUARE"
        }
    }

def build_indent_request(start_index, end_index, nesting_level):
    """
    Indent the bullet by nesting_level. Each nesting level ~ 36 PT.
    """
    return {
        "updateParagraphStyle": {
            "range": {
                "startIndex": start_index,
                "endIndex": end_index
            },
            "paragraphStyle": {
                "indentStart": {
                    "magnitude": 36 * nesting_level,
                    "unit": "PT"
                }
            },
            "fields": "indentStart"
        }
    }

def build_italic_footer_request(start_index, end_index):
    """
    Italic + smaller font for the footer lines (e.g., "Meeting recorded by:" lines).
    """
    return {
        "updateTextStyle": {
            "range": {
                "startIndex": start_index,
                "endIndex": end_index
            },
            "textStyle": {
                "italic": True,
                "fontSize": {"magnitude": 10, "unit": "PT"}
            },
            "fields": "italic,fontSize"
        }
    }

def build_mention_style_requests(line_text, text_start_index):
    """
    Finds all mentions (@someone) in line_text, returns a list of updateTextStyle
    requests that apply a distinct color and bold style.
    
    text_start_index = the doc index where this line's text begins.
    """
    mention_pattern = re.compile(r"(@[A-Za-z0-9_]+)")
    requests = []
    
    for match in mention_pattern.finditer(line_text):
        start_pos = text_start_index + match.start()
        end_pos = text_start_index + match.end()
        requests.append({
            "updateTextStyle": {
                "range": {
                    "startIndex": start_pos,
                    "endIndex": end_pos
                },
                "textStyle": {
                    # Distinct color (orange) + bold
                    "foregroundColor": {
                        "color": {
                            "rgbColor": {
                                "red": 1.0,
                                "green": 0.5,
                                "blue": 0.0
                            }
                        }
                    },
                    "bold": True
                },
                "fields": "foregroundColor,bold"
            }
        })
    return requests

# ===========================================================================
# 4) Main parser: read markdown line-by-line and build a FORWARD-ORDER request list.
# ===========================================================================
def parse_markdown_to_requests(markdown_text, start_index=1):
    """
    Convert the markdown text into a list of docs batchUpdate requests, 
    without reversing them. We track current_index as we insert text.
    """
    requests = []
    current_index = start_index

    lines = markdown_text.split("\n")
    for line in lines:
        # Skip completely empty lines
        if not line.strip():
            continue
        
        stripped_line = line.strip()

        # HEADINGS
        if re.match(r"^# ", stripped_line):
            # Heading 1
            text = stripped_line[2:].strip()
            insert_req, new_idx = build_insert_text_request(text, current_index)
            requests.append(insert_req)

            # Style as HEADING_1
            requests.append(build_update_paragraph_style_request(
                current_index, new_idx, "HEADING_1"
            ))

            # Mention styling (if any)
            requests.extend(build_mention_style_requests(text, current_index))

            current_index = new_idx

        elif re.match(r"^## ", stripped_line):
            # Heading 2
            text = stripped_line[3:].strip()
            insert_req, new_idx = build_insert_text_request(text, current_index)
            requests.append(insert_req)

            # Style as HEADING_2
            requests.append(build_update_paragraph_style_request(
                current_index, new_idx, "HEADING_2"
            ))

            # Mention styling
            requests.extend(build_mention_style_requests(text, current_index))

            current_index = new_idx

        elif re.match(r"^### ", stripped_line):
            # Heading 3
            text = stripped_line[4:].strip()
            insert_req, new_idx = build_insert_text_request(text, current_index)
            requests.append(insert_req)

            # Style as HEADING_3
            requests.append(build_update_paragraph_style_request(
                current_index, new_idx, "HEADING_3"
            ))

            # Mention styling
            requests.extend(build_mention_style_requests(text, current_index))

            current_index = new_idx

        # CHECKBOXES: - [ ] or * [ ]
        elif re.match(r"^- \[ \]", stripped_line) or re.match(r"^\* \[ \]", stripped_line):
            text_line = re.sub(r"^(-|\*) \[ \]", "", stripped_line).strip()
            nesting_level = extract_nesting_level(line)

            # Insert the line
            insert_req, new_idx = build_insert_text_request(text_line, current_index)
            requests.append(insert_req)

            # Mention styling
            requests.extend(build_mention_style_requests(text_line, current_index))

            # Make it a checkbox bullet
            requests.append(build_bullet_request(current_index, new_idx, checkbox=True))

            # Indent if nesting level > 0
            if nesting_level > 0:
                requests.append(build_indent_request(current_index, new_idx, nesting_level))

            current_index = new_idx

        # NORMAL BULLETS: starts with "-" or "*"
        elif re.match(r"^[-*]", stripped_line):
            # For lines that are not checkboxes
            text_line = re.sub(r"^[-*]", "", stripped_line).strip()
            nesting_level = extract_nesting_level(line)

            # Insert the line
            insert_req, new_idx = build_insert_text_request(text_line, current_index)
            requests.append(insert_req)

            # Mention styling
            requests.extend(build_mention_style_requests(text_line, current_index))

            # Create normal bullet
            requests.append(build_bullet_request(current_index, new_idx, checkbox=False))

            # Indent if nesting level > 0
            if nesting_level > 0:
                requests.append(build_indent_request(current_index, new_idx, nesting_level))

            current_index = new_idx

        else:
            # Regular text lines
            insert_req, new_idx = build_insert_text_request(stripped_line, current_index)
            requests.append(insert_req)

            # Mention styling
            requests.extend(build_mention_style_requests(stripped_line, current_index))

            # If it's the footer lines, style them differently (italic, smaller)
            if stripped_line.startswith("Meeting recorded by:") or stripped_line.startswith("Duration:"):
                requests.append(build_italic_footer_request(current_index, new_idx))

            current_index = new_idx

    return requests

# ===========================================================================
# 5) Create the doc, parse requests in forward order, then batchUpdate.
# ===========================================================================
doc_title = "Product Team Sync (Auto-Generated) with Mentions"
document_id = create_new_document(doc_title)
if not document_id:
    print("Failed to create a new document.")
    sys.exit(1)

# Build the forward-ordered requests
requests_forward = parse_markdown_to_requests(MARKDOWN_TEXT)

try:
    docs_service.documents().batchUpdate(
        documentId=document_id,
        body={"requests": requests_forward}
    ).execute()
    print(f"Successfully updated document with ID: {document_id}")
    print(f"View it here: https://docs.google.com/document/d/{document_id}/edit")
except HttpError as e:
    print("Error applying batchUpdate:", e)
